# imports

In [1]:
import sys
sys.path.append('../src/')
import json
import os
from tqdm import tqdm
import numpy as np
from concurrent.futures import ProcessPoolExecutor
from sentencepiece import SentencePieceProcessor
from functools import partial

# paths

In [2]:
data = '../llama.c-/data/TinyStories_all_data/'
#ta = '/home/cindy/learning/karpathy/llms/llama2.c/data/TinyStories_all_data/'

In [3]:
tokenizer_path = '../tokenizer.model'

In [4]:
out_data = '../instruct_dataset/'
os.makedirs(out_data, exist_ok=True)

# preprocessing functions

In [5]:
def replace_words(
    prompt, 
    words_list
):
    for words_to_remove, replacement in words_list:
        prompt = prompt.replace(words_to_remove, replacement)
    return prompt
        

In [6]:
def preprocess_prompt(prompt, suffix=' Possible story:'):
    words_list = [
    ("Write a short story (3-5 paragraphs) which only uses very simple words that a 3 year old child would understand.", "Write a story."),
    (" Remember to only use simple words!", ""),
    ("\n\nPossible story:", ""),
    ("try to at some point use", "try to use")
    ]
    prompt = replace_words(prompt, words_list)
    return prompt + suffix

# tokenize and save

In [7]:
def save(out_folder, filename, data_to_save):
    filepath = os.path.join(out_folder, filename)
    print(out_folder, filename, data_to_save.shape)
    with open(filepath, "wb") as f:
        f.write(data_to_save.tobytes())
    print('Saved to ', filepath)

In [8]:
def tokenize_chunk(chunk_path, out_folder, tokenizer, max_seq_len, pad_token):
    with open(chunk_path, 'r') as f:
        chunk = json.load(f)
        print('Tokenize ', chunk_path)
    all_tokens = []
    all_labels = []
    for sample in tqdm(chunk):
        story = sample['story'].strip()
        prompt = preprocess_prompt(sample['instruction']['prompt:'].strip())
        tokenized_prompt = tokenizer.encode(prompt)
        
        prompt_and_story = tokenized_prompt + [tokenizer.bos_id()] + tokenizer.encode(story) + [tokenizer.eos_id()]
        label = [pad_token]*len(tokenized_prompt) + [tokenizer.bos_id()] + tokenizer.encode(story) + [tokenizer.eos_id()]

        if len(prompt_and_story) <= max_seq_len:
            prompt_and_story += [pad_token] * (max_seq_len - len(prompt_and_story))
            label += [pad_token] * (max_seq_len - len(label))
            assert len(prompt_and_story) == len(label) == max_seq_len
            all_tokens.extend(prompt_and_story)
            all_labels.extend(label)

    all_tokens = np.array(all_tokens, dtype=np.int16)
    all_labels = np.array(all_labels, dtype=np.int16)
    
    all_tokens_filename = chunk_path.split('/')[-1].replace('.json', '.bin')
    save(out_folder=out_folder, filename=all_tokens_filename, data_to_save=all_tokens)

    all_labels_filename = all_tokens_filename.replace('data', 'labels')
    save(out_folder=out_folder, filename=all_labels_filename, data_to_save=all_labels)

        
    
def tokenize_all_chunks(data, out_folder, tokenizer, max_seq_len, pad_token, max_workers=5):
    tokenize_chunk_fn = partial(
        tokenize_chunk,
        out_folder=out_folder,
        tokenizer=tokenizer,
        max_seq_len=max_seq_len,
        pad_token=pad_token
    )
        
    tokenize_chunk_paths = [os.path.join(data, fn) for fn in os.listdir(data) if fn.endswith('.json')]
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        executor.map(tokenize_chunk_fn, tokenize_chunk_paths)
        
        

In [9]:
tokenizer = SentencePieceProcessor(model_file=tokenizer_path)

In [10]:
tokenize_all_chunks(
    data=data, 
    out_folder=out_data, 
    tokenizer=tokenizer, 
    max_seq_len=350,
    pad_token=-100
)

Tokenize  ../llama.c-/data/TinyStories_all_data/data34.json


  1%|          | 1164/100000 [00:01<01:33, 1060.84it/s]

Tokenize  ../llama.c-/data/TinyStories_all_data/data07.json


  1%|▏         | 1277/100000 [00:01<01:31, 1079.22it/s]

Tokenize  ../llama.c-/data/TinyStories_all_data/data04.json


...

 ../instruct_dataset/data44.bin
../instruct_dataset/ labels44.bin (32884600,)


 91%|█████████ | 90996/100000 [01:11<00:07, 1271.94it/s]

Saved to  ../instruct_dataset/labels44.bin


100%|██████████| 100000/100000 [01:17<00:00, 1282.92it/s]


../instruct_dataset/ data31.bin (32855900,)
Saved to  ../instruct_dataset/data31.bin
../instruct_dataset/ labels31.bin (32855900,)
Saved to  ../instruct_dataset/labels31.bin
../instruct_dataset/ data35.bin (32884250,)
Saved to  ../instruct_dataset/data35.bin
../instruct_dataset/ labels35.bin (32884250,)
Saved to  ../instruct_dataset/labels35.bin
../instruct_dataset/ data08.bin (32878650,)
Saved to  ../instruct_dataset/data08.bin
../instruct_dataset/ labels08.bin (32878650,)
Saved to  ../instruct_dataset/labels08.bin


# check data

In [11]:


with open(out_data + 'data11.bin', 'rb') as f:
    x = f.read()
x = np.frombuffer(x, dtype=np.int16)



In [12]:
with open(out_data + 'labels11.bin', 'rb') as f:
    y = f.read()
y = np.frombuffer(y, dtype=np.int16)